## IBM Coursera Capstone Project

This is the capstone project for the IBM Coursera Professional Certificate

In [123]:
import pandas as pd
import numpy as np
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [124]:
import requests
from bs4 import BeautifulSoup

Request the html from the wikipedia website and read it into a BeatifulSoup object

In [125]:
result = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [126]:
bs = BeautifulSoup(result.content,"lxml")

Loop through the table and append each of the rows to a string  
Put it into a dataframe and filter out the postal codes that dont have a Borough assigned

In [127]:
import re
bs_table = bs.find("tbody")

data = [] 
for row in bs_table.find_all("tr"):
    data.append([r.text.strip() for r in row.find_all("td")])
                 
d = pd.DataFrame(data[1:],columns=["postal_code","borough","neighborhood"])
d = d.loc[d.borough != "Not assigned"]
d

,postal_code,borough,neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


If a postal code belongs to multiple neighborhoods then append the neighborhoods with a comma. We are assuming that each postal code only belongs to 1 borough here. 

In [128]:
d = d.groupby(["postal_code","borough"])["neighborhood"].apply(lambda x: ",".join(x)).reset_index()

#Confirming that the neighborhoods were joined succesffuly
d.loc[d.postal_code == "M5A"]

,postal_code,borough,neighborhood
53,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In case there are blank neighborhoods, set them to the borough. It doesnt appear that any of the data contains this instance however this was still done for future iterations.

In [129]:
d["neighborhood"] = d.apply(lambda x: x.borough if x.neighborhood == "" else x.neighborhood, axis = 1)

In [130]:
d.shape

(103, 3)